In [ ]:
""" 
1- Preciso excluir as linhas de 'nível' 'SECUNDARIA'. Acabei mandando a base toda pra rodar. 
"""

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

df_main = pd.read_csv('./csv/ESTANDAR_BASICA_I2324.csv')
df_scraping = pd.read_csv('./csv/base_scraping.csv')
extraction = pd.read_csv('./csv/extraction.csv')
df_high_mx = pd.read_csv('./csv/high_touch_mx.csv')
ladas_mexico = pd.read_csv('./csv/ladas_mexico.csv')

In [ ]:
display(df_high_mx.head(1))

In [ ]:
# Faz o merge adicionando a coluna 'telefone_scraping' à base de 12k

# Eliminar duplicatas de clave cct
df_scraping_unique = df_scraping.drop_duplicates(subset='clavecct', keep='first')

df_high_mx = extraction.merge(
    df_scraping_unique[['clavecct', 'telefone_scraping']],
    on='clavecct',
    how='left'
)

In [ ]:
# Verificar quantos números foram adicionados através da contagem de ocorrências válidas para a coluna 'telefone_scraping'.

tel_adicionados = df_high_mx[df_high_mx['telefone_scraping'].notna()]   
porcentagem = (len(tel_adicionados) / len(df_high_mx)) * 100 
print(f'O dataframe tem {len(df_high_mx)} linhas.')                    
print(f'Foram adicionados {len(tel_adicionados)} números de telefone novos ({porcentagem:.1f}%).')

In [ ]:
import phonenumbers

# Cria uma cópia para evitar warning
tel_validos = df_high_mx[df_high_mx['telefone_scraping'].notna()].copy()

def validar_telefone(numero, codigo_pais='MX'):
    try:
        parsed = phonenumbers.parse(numero, codigo_pais)
        return phonenumbers.is_valid_number(parsed)
    except phonenumbers.NumberParseException:
        return False

# Aplica a validação de forma segura
tel_validos['telefone_valido'] = tel_validos['telefone_scraping'].apply(
    lambda x: validar_telefone(x, 'MX')
)

# Exibe os resultados
print(f'O dataframe tem {len(df_high_mx)} escolas.') 

print(f'Total de telefones adicionados: {len(tel_validos)}')
print("Total de telefones válidos:", tel_validos['telefone_valido'].sum())

proporcao_total_real = (tel_validos['telefone_valido'].sum() / len(df_high_mx)) * 100
print(f'Proporção de telefones válidos adicionados: {proporcao_total_real:.1f}%')

proporcao = tel_validos['telefone_valido'].mean() * 100
print(f"Proporção de válidos em relação a inválidos: {proporcao:.1f}%")

telefones_invalidos = tel_validos[~tel_validos['telefone_valido']]
print(f'Quantidade de telefones inválidos: {len(telefones_invalidos)}')

In [ ]:
print('Telefones inválidos abaixo:')
display(telefones_invalidos['telefone_scraping'])

In [ ]:
# Filtra os inválidos
telefones_invalidos = tel_validos[~tel_validos['telefone_valido']].copy()

# Cria coluna com o comprimento dos números
telefones_invalidos['qtde_caracteres'] = telefones_invalidos['telefone_scraping'].str.len()

# Agrupa e conta
resumo = telefones_invalidos.groupby('qtde_caracteres').size().reset_index(name='quantidade')

display(resumo)


In [ ]:
display(df_high_mx.head())

In [ ]:
print(f'O dataframe ladas_mexico tem {len(ladas_mexico)} linhas.')
print(f'Temos {len(ladas_mexico['lada'].unique())} ladas únicas.')
print(f'Temos {len(ladas_mexico['ciudad'].unique())} cidades únicas.')
print(f'Temos {len(ladas_mexico['estado'].unique())} estados únicos.')

In [ ]:
# Esta célula identifica todas as combinações de cidade + estado que possuem mais de uma clave LADA diferente.
# Primeiro, agrupa os dados por cidade e estado e conta quantas LADAs únicas existem em cada grupo.
# Em seguida, filtra apenas os pares com mais de uma LADA.
# Depois, faz um merge com outro agrupamento que coleta todas as LADAs únicas por par cidade + estado (em forma de lista),
# resultando em um DataFrame com as colunas: cidade, estado e a lista de LADAs correspondentes.
# Por fim, exibe quantas dessas combinações existem e mostra o DataFrame resultante.

claves_por_par = (
    ladas_mexico
    .groupby(['ciudad', 'estado'])['lada']
    .nunique()
    .reset_index()
    .query('lada > 1')
    .merge(
        ladas_mexico.groupby(['ciudad', 'estado'])['lada'].unique().reset_index().rename(columns={'lada': 'claves_lada'}),
        on=['ciudad', 'estado']
    )[['ciudad', 'estado', 'claves_lada']]
)

print(f'Temos {len(claves_por_par)} combinações de cidade + estado com claves lada diferentes.')
display(claves_por_par)
display(ladas_mexico.head())

In [ ]:
import pandas as pd
import numpy as np
import phonenumbers

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

df_high_mx = pd.read_csv('./csv/high_touch_mx.csv')
ladas_mexico = pd.read_csv('./csv/ladas_mexico.csv')

# Esta célula identifica todas as combinações de cidade + estado que possuem mais de uma clave LADA diferente.
# Primeiro, agrupa os dados por cidade e estado e conta quantas LADAs únicas existem em cada grupo.
# Em seguida, filtra apenas os pares com mais de uma LADA.
# Depois, faz um merge com outro agrupamento que coleta todas as LADAs únicas por par cidade + estado (em forma de lista),
# resultando em um DataFrame com as colunas: cidade, estado e a lista de LADAs correspondentes.
# Por fim, exibe quantas dessas combinações existem e mostra o DataFrame resultante.

claves_por_par = (
    ladas_mexico
    .groupby(['ciudad', 'estado'])['lada']
    .nunique()
    .reset_index()
    .query('lada > 1')
    .merge(
        ladas_mexico.groupby(['ciudad', 'estado'])['lada'].unique().reset_index().rename(columns={'lada': 'claves_lada'}),
        on=['ciudad', 'estado']
    )[['ciudad', 'estado', 'claves_lada']]
)

#print(f'Temos {len(claves_por_par)} combinações de cidade + estado com claves lada diferentes.')
#display(claves_por_par)
#display(ladas_mexico.head())

# Função de validação
def validar_telefone(numero: str, codigo_pais='MX'):
    try:
        parsed = phonenumbers.parse(numero, codigo_pais)
        if phonenumbers.is_valid_number(parsed):
            return numero
    except phonenumbers.NumberParseException:
        pass
    return None

# Normalização prévia dos DataFrames auxiliares
claves_por_par["estado"] = claves_por_par["estado"].astype(str).str.strip().str.lower()
claves_por_par["ciudad"] = claves_por_par["ciudad"].astype(str).str.strip().str.lower()

ladas_mexico["estado"] = ladas_mexico["estado"].astype(str).str.strip().str.lower()
ladas_mexico["ciudad"] = ladas_mexico["ciudad"].astype(str).str.strip().str.lower()

# Função aplicada por linha
def processar_linha(row, claves_por_par, ladas_mexico):
    telefone = row.get("telefone_scraping")
    estado = str(row.get("n_entidad", "")).strip().lower()
    cidade = str(row.get("city", "")).strip().lower()
    resultados = {}

    if isinstance(telefone, str) and len(telefone) in (7, 8):
        # Primeiro: tenta encontrar na claves_por_par
        mask_cp = (claves_por_par["estado"] == estado) & (claves_por_par["ciudad"] == cidade)
        rec_cp = claves_por_par[mask_cp]

        if not rec_cp.empty:
            ddds = rec_cp.iloc[0]["claves_lada"]
            for i, ddd in enumerate(ddds):
                numero = str(ddd) + telefone
                validado = validar_telefone(numero)
                if validado:
                    col = "tel_ddd_adicionado" if i == 0 else f"tel_ddd_adicionado_{i+1}"
                    resultados[col] = validado
            return pd.Series(resultados)

        # Caso contrário: tenta encontrar na ladas_mexico
        mask_lm = (ladas_mexico["estado"] == estado) & (ladas_mexico["ciudad"] == cidade)
        rec_lm = ladas_mexico[mask_lm]

        if not rec_lm.empty:
            ddd = rec_lm.iloc[0]["lada"]
            numero = str(ddd) + telefone
            validado = validar_telefone(numero)
            if validado:
                resultados["tel_ddd_adicionado"] = validado

    return pd.Series(resultados)

# Função principal
def adicionar_ddd(df_high_mx, claves_por_par, ladas_mexico):
    novos_ddds = df_high_mx.apply(
        processar_linha,
        axis=1,
        args=(claves_por_par, ladas_mexico)
    )
    df_ddd_adicionado = pd.concat([df_high_mx, novos_ddds], axis=1)
    return df_ddd_adicionado

# Execução
df_ddd_adicionado = adicionar_ddd(df_high_mx, claves_por_par, ladas_mexico)

# Filtra as colunas que contém 'tel_ddd_adicionado' no nome
colunas_ddd = df_ddd_adicionado.filter(like='tel_ddd_adicionado')

# Quantidade de colunas com 'tel_ddd_adicionado'
num_colunas = len(colunas_ddd.columns)

# Soma total de valores válidos (não nulos) em todas essas colunas
total_validos = colunas_ddd.notna().sum().sum()

print(f"Colunas 'tel_ddd_adicionado' criadas: {num_colunas}")
print(f"Total de valores válidos somados em todas as colunas: {total_validos}")

# Filtra linhas onde a primeira coluna 'tel_ddd_adicionado' tem valor válido (não nulo)
primeira_coluna = colunas_ddd.columns[0]
filtro_primeira_coluna = df_ddd_adicionado[primeira_coluna].notna()

# Retorna o dataframe completo filtrado por esse critério
df_filtrado = df_ddd_adicionado[filtro_primeira_coluna]

display(df_filtrado.head())


Temos 39 combinações de cidade + estado com claves lada diferentes.
Colunas 'tel_ddd_adicionado' criadas: 1
Total de valores válidos somados em todas as colunas: 142


,id_escola_fuzzy,tem_inicial,tem_preescolar,tem_primaria,tem_secundaria,matricula_inicial,matricula_preescolar,matricula_primaria,matricula_secundaria,matricula_inf_fund,matricula_total,city,n_entidad,name,address,doc_tot,clavecct,docentes_por_aluno,actor_uuid,address_name,similar_address_name,similar_score,similar_id,include_words,telefone_scraping,tel_ddd_adicionado
60,2010 BI-CENTENARIO - SALAMANCA - GUANAJUATO,0,0,1,0,0,0,49,0,49,49,salamanca,GUANAJUATO,2010 BI-CENTENARIO,CALLE ALLENDE,6,11PPR1031H,8.166667,NaN,calle allende 2010 bi-centenario,calle agua dulcebellavista riama,44.0,0ea21c31bafae4ae12a773aa64dc6ccf5453c2b42351652617b613ad,0,6486080,4646486080
88,ACADEMIA INTERNACIONAL SAN MIGUEL ALLENDE - SAN MIGUEL DE ALLENDE - GUANAJUATO,0,0,1,1,0,0,108,37,108,145,san miguel de allende,GUANAJUATO,ACADEMIA INTERNACIONAL SAN MIGUEL ALLENDE,CALLE STIRLING DICKINSON,6,11PES0460L,24.166667,NaN,calle stirling dickinson academia internacional san miguel allende,calle stirling dickinsonsan antonio academia internacional san miguel de allende,90.0,a7864f8e13a445a242891c12c877f4e5e9b0f2ede381c3d7f59742f4,0,1523235,4151523235
89,ACADEMIA INTERNACIONAL SAN MIGUEL DE ALLENDE - SAN MIGUEL DE ALLENDE - GUANAJUATO,0,1,0,0,0,43,0,0,43,43,san miguel de allende,GUANAJUATO,ACADEMIA INTERNACIONAL SAN MIGUEL DE ALLENDE,CALLE STIRLING DICKINSON,3,11PJN1101W,14.333333,NaN,calle stirling dickinson academia internacional san miguel de allende,calle stirling dickinsonsan antonio academia internacional san miguel de allende,93.0,a7864f8e13a445a242891c12c877f4e5e9b0f2ede381c3d7f59742f4,0,1523235,4151523235
106,ADRIAN DIAZ CORDOBA - CELAYA - GUANAJUATO,0,1,1,1,0,35,73,36,108,144,celaya,GUANAJUATO,ADRIAN DIAZ CORDOBA,CALLE ACAPULCO,9,11PJN0176F,16.000000,NaN,calle acapulco adrian diaz cordoba,calle acapulcoel zapote adrian diaz cordoba,88.0,1534da836e797614b804f9b19dd7e3773b5376a17c063620713be43d,0,6134819,4616134819
171,ALEJANDRO MAGNO - SALAMANCA - GUANAJUATO,0,1,1,1,0,65,137,75,202,277,salamanca,GUANAJUATO,ALEJANDRO MAGNO,CAMINO A MANCERA,13,11PJN1139I,21.307692,NaN,camino a mancera alejandro magno,camino a manceraarboledas de ciudad bajio alejandro magno,72.0,e1be0bb2a0731127556d62992aec034557e63119627be179164bb5c6,0,6903000,4646903000


In [ ]:
# Filtra as colunas que contém 'tel_ddd_adicionado' no nome
colunas_ddd = df_ddd_adicionado.filter(like='tel_ddd_adicionado')

# Quantidade de colunas com 'tel_ddd_adicionado'
num_colunas = len(colunas_ddd.columns)

# Soma total de valores válidos (não nulos) em todas essas colunas
total_validos = colunas_ddd.notna().sum().sum()

print(f"Colunas 'tel_ddd_adicionado' criadas: {num_colunas}")
print(f"Total de valores válidos somados em todas as colunas: {total_validos}")

# Filtra linhas onde a primeira coluna 'tel_ddd_adicionado' tem valor válido (não nulo)
primeira_coluna = colunas_ddd.columns[0]
filtro_primeira_coluna = df_ddd_adicionado[primeira_coluna].notna()

# Retorna o dataframe completo filtrado por esse critério
df_filtrado = df_ddd_adicionado[filtro_primeira_coluna]

display(df_filtrado)
